In [1]:
# 셀 1: 필요한 라이브러리 임포트
import requests
import xml.etree.ElementTree as ET
import json

# 셀 2: 청년 정책 검색 함수 정의 (전체 필드 추출 포함, 페이지네이션 추가)
def get_all_youth_policies(api_key, display=100):
    url = "https://www.youthcenter.go.kr/opi/youthPlcyList.do"
    all_policies = []
    page_index = 1
    
    while True:
        params = {
            "openApiVlak": api_key,
            "display": display,
            "pageIndex": page_index,
        }
        
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            try:
                # XML 데이터를 파싱
                root = ET.fromstring(response.content)
                policies = []
                
                for policy in root.findall('youthPolicy'):
                    policy_data = {}
                    
                    # 모든 자식 노드의 태그와 텍스트 값을 추출
                    for child in policy:
                        policy_data[child.tag] = child.text
                    
                    policies.append(policy_data)
                
                if not policies:
                    break  # 더 이상 데이터가 없으면 종료
                
                all_policies.extend(policies)
                page_index += 1
                
            except ET.ParseError:
                print("Error: Failed to parse XML response.")
                break
        else:
            print(f"Error: {response.status_code} - {response.text}")
            break
    
    return all_policies

# 셀 3: 결과를 파일로 저장하는 함수
def save_to_file(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"Data successfully saved to {filename}")

# 셀 4: 함수 사용 예시 및 파일 저장
api_key = "bb7d5124a944a089bc868488"  # Replace with your actual API key
result = get_all_youth_policies(api_key, display=100)

# 저장할 파일명 설정
filename = "all_youth_policies.json"
save_to_file(result, filename)


Data successfully saved to all_youth_policies.json
